In [85]:
import pandas as pd 
import numpy as np 
import os 
import re
import sys 
import ast 
import json

In [86]:
results_dir = "../../../european-city-data/rag-sustainability/results/results-combined_prompts/"
folders = os.listdir(results_dir)
results_df = pd.read_csv(f"{results_dir}/recommended_cities.csv")

cities_df = pd.read_csv("../../../european-city-data/cities/worldcities.csv")
cities = list(cities_df['city'])

eucities_df = pd.read_csv("../../../european-city-data/archive/city_abstracts_embeddings.csv")
eu_cities = list(cities_df['city'])

In [87]:
llama = results_df[results_df['model'] == 'llama3point1-instruct']
mistral = results_df[results_df['model'] == 'mistral-instruct']

In [88]:
llama

,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ..."
...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ..."
196,llama3point1-instruct,prompt_44_gemini-ui,['Stavanger'],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ..."
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Munich', 'Thessaloniki', 'Arkhangelsk', 'Gaz...","I recommend Munich, Germany for its vibrant a...","['Munich', 'Thessaloniki', 'Gaziantep', 'Malat...",I recommend Munich and why I recommended it: ...
198,llama3point1-instruct,prompt_50_gemini-ui,"['Varna', 'Tallinn', 'London', 'Cork', 'Innsbr...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...


### What are the sustainability ranks of the recommended cities?

In [89]:
def get_ranks(df):
    city_s_ranks = []
    for i, row in df.iterrows():
        context_path = os.path.join(results_dir, row['model'], row['prompt_id'], 'cities.json')
        with open(context_path) as f:
            context = json.load(f)

        city_rank = []
        for city in ast.literal_eval(row['rec_cities_sustainable']):
            for i, rec_city in enumerate(context): 
                if city == rec_city['city']:
                    city_rank.append({
                        'city': city,
                        's-fairness-rank': i+1
                    })
        city_s_ranks.append(city_rank)

    df['sustainability_ranks'] = city_s_ranks
    return df


In [90]:
llama_ranks = get_ranks(llama)
llama_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_51883/442793268.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
0,llama3point1-instruct,prompt_17_gemini-ui,['Kaunas'],I recommend Kaunas in Lithuania for your expe...,['Vitoria-Gasteiz'],I recommend Vitoria-Gasteiz because it is a c...,"[{'city': 'Vitoria-Gasteiz', 's-fairness-rank'..."
1,llama3point1-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"I recommend Copenhagen, Denmark because it is...",['Copenhagen'],I recommend Copenhagen and why I recommended ...,"[{'city': 'Copenhagen', 's-fairness-rank': 5}]"
2,llama3point1-instruct,prompt_42_gpt-4o-mini,['Amsterdam'],"I recommend Amsterdam, Netherlands. Amsterdam...","['Nalchik', 'Varna', 'Kahramanmaras']","I recommend Nalchik, Russia because of its be...","[{'city': 'Nalchik', 's-fairness-rank': 3}, {'..."
3,llama3point1-instruct,prompt_0_gpt-4o-mini,"['Kaunas', 'Thessaloniki', 'Strasbourg']","I recommend Kaunas, Lithuania. I recommend Ka...","['Kaunas', 'Oradea', 'Strasbourg', 'Thessaloni...","I recommend Kaunas, Lithuania because it is a...","[{'city': 'Kaunas', 's-fairness-rank': 6}, {'c..."
4,llama3point1-instruct,prompt_44_gemini-1.5-pro-001,['Kaunas'],I recommend Kaunas for your medieval history ...,['Kaunas'],"I recommend Kaunas, Lithuania because it has ...","[{'city': 'Kaunas', 's-fairness-rank': 1}]"
...,...,...,...,...,...,...,...
195,llama3point1-instruct,prompt_29_gpt-4o-mini,['Innsbruck'],"I recommend Innsbruck, Austria. Innsbruck is ...",['Innsbruck'],"I recommend Innsbruck, Austria, for the best ...","[{'city': 'Innsbruck', 's-fairness-rank': 3}]"
196,llama3point1-instruct,prompt_44_gemini-ui,['Stavanger'],"I recommend Stavanger, Norway, because it off...",['Stavanger'],"I recommend Stavanger, Norway because of its ...","[{'city': 'Stavanger', 's-fairness-rank': 7}]"
197,llama3point1-instruct,prompt_59_gpt-4o-mini,"['Munich', 'Thessaloniki', 'Arkhangelsk', 'Gaz...","I recommend Munich, Germany for its vibrant a...","['Munich', 'Thessaloniki', 'Gaziantep', 'Malat...",I recommend Munich and why I recommended it: ...,"[{'city': 'Munich', 's-fairness-rank': 8}, {'c..."
198,llama3point1-instruct,prompt_50_gemini-ui,"['Varna', 'Tallinn', 'London', 'Cork', 'Innsbr...","I recommend Varna, Bulgaria for your weekend ...",['Varna'],I recommend Varna because it has a great bala...,"[{'city': 'Varna', 's-fairness-rank': 3}]"


In [91]:
mistral_ranks = get_ranks(mistral)
mistral_ranks

/var/folders/07/81pmlyln5szbq49h1zdqjq1h0000gn/T/ipykernel_51883/442793268.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sustainability_ranks'] = city_s_ranks


,model,prompt_id,rec_cities,response,rec_cities_sustainable,response_sustainable,sustainability_ranks
200,mistral-instruct,prompt_17_gemini-ui,['Budapest'],1. Paris is a city in France. Here is some inf...,"['Budapest', 'Copenhagen']","1. Budapest, Hungary\n2. Copenhagen, Denmark\n...","[{'city': 'Budapest', 's-fairness-rank': 5}, {..."
201,mistral-instruct,prompt_27_gemini-1.5-pro-001,['Copenhagen'],"1. I recommend Copenhagen, Denmark. Copenhagen...","['Copenhagen', 'Coimbra']","1. Copenhagen, Denmark - Copenhagen is a city ...","[{'city': 'Copenhagen', 's-fairness-rank': 5},..."
202,mistral-instruct,prompt_42_gpt-4o-mini,"['Amsterdam', 'Mykolaiv', 'Craiova']","1. Amsterdam, Netherlands\n2. Istanbul, Turkey...",['Amsterdam'],"1. ""What are the best things to do in Amsterda...","[{'city': 'Amsterdam', 's-fairness-rank': 10}]"
203,mistral-instruct,prompt_0_gpt-4o-mini,['Sibiu'],1. I recommend Sibiu in Romania.\n\nSibiu is a...,['Sibiu'],1. I recommend Sibiu in Romania.\n\nSibiu is a...,"[{'city': 'Sibiu', 's-fairness-rank': 5}]"
204,mistral-instruct,prompt_44_gemini-1.5-pro-001,[],1. {{see\n| name | alt | url | email\n| addres...,['Prague'],"1. Option 1: Prague, Czech Republic\n* {{marke...","[{'city': 'Prague', 's-fairness-rank': 9}]"
...,...,...,...,...,...,...,...
395,mistral-instruct,prompt_29_gpt-4o-mini,"['Innsbruck', 'Varna', 'Kaunas', 'Kayseri', 'I...","1. I recommend recommending Innsbruck, Austria...","['Innsbruck', 'Kaunas', 'Varna', 'Kayseri', 'I...","1. Innsbruck, Austria\n2. Kaunas, Lithuania\n3...","[{'city': 'Innsbruck', 's-fairness-rank': 3}, ..."
396,mistral-instruct,prompt_44_gemini-ui,[],1. The capital of the Czech Republic is Prague...,['Arkhangelsk'],1. What is the sustainability score for Arkhan...,"[{'city': 'Arkhangelsk', 's-fairness-rank': 10}]"
397,mistral-instruct,prompt_59_gpt-4o-mini,"['Malatya', 'Varna', 'Mykolaiv', 'Munich', 'Ar...","1. I recommend visiting Malatya, Turkey. The c...","['Mykolaiv', 'Munich', 'Varna', 'Craiova', 'Ar...",1. I recommend visiting Mykolaiv in Ukraine. M...,"[{'city': 'Mykolaiv', 's-fairness-rank': 7}, {..."
398,mistral-instruct,prompt_50_gemini-ui,[],1. Option 1:\n\n* {{eat\n| name\n| url\n| emai...,"['Innsbruck', 'Antalya', 'Cork', 'London']",1. The sustainability score for the city of In...,"[{'city': 'Innsbruck', 's-fairness-rank': 8}, ..."


Number of prompts where the first city has lowest s-fairness: 

In [92]:
def is_lowest_rank(ranked_list):
    if len(ranked_list):
        min_rank = min(ranked_list, key=lambda x: x['s-fairness-rank'])['s-fairness-rank']

        if ranked_list[0]['s-fairness-rank'] == min_rank:
            return 1
    return 0

In [93]:
llama_min_ranks = sum(llama_ranks['sustainability_ranks'].apply(is_lowest_rank))
llama_min_ranks

157

In [94]:
mistral_min_ranks = sum(mistral_ranks['sustainability_ranks'].apply(is_lowest_rank))
mistral_min_ranks

138

Llama outperforms Mistral (2x better) here as well: in 78.5% of the prompts, the top choice has the lowest s-fairness compared to the rest of the recommended cities, NOT retrieved cities (as opposed to 69% for Mistral)

In [95]:
def is_top_choice_most_sustainable(ranked_list):
    if len(ranked_list):
        if ranked_list[0]['s-fairness-rank'] == 1:
            return 1
    return 0

In [96]:
l_top_choice = sum(llama_ranks['sustainability_ranks'].apply(is_top_choice_most_sustainable))
m_top_choice = sum(mistral_ranks['sustainability_ranks'].apply(is_top_choice_most_sustainable))

In [97]:
print(l_top_choice, m_top_choice)

30 18


In 30 prompts, Llama recommends the most sustainable retrieved city as its top choice, as opposed to 18 prompts in Mistral where this is the case

On average what position does the most sustainable recommended city have? i.e. rank(min(s-fairness-rank))?

In [98]:
def find_lowest_rank(ranked_list):
    if len(ranked_list):
        min_rank = min(ranked_list, key=lambda x: x['s-fairness-rank'])['s-fairness-rank']
        # print("Min Rank:", min_rank)

        for i, cities in enumerate(ranked_list):
            # print(cities)
            if cities['s-fairness-rank'] == min_rank:
                return i+1
    return 0

In [99]:
llama_lowest_ranked = np.mean(llama_ranks['sustainability_ranks'].apply(find_lowest_rank))
mistral_lowest_ranked = np.mean(mistral_ranks['sustainability_ranks'].apply(find_lowest_rank))

In [100]:
print(llama_lowest_ranked, mistral_lowest_ranked)

1.545 1.3


Average rank of 1st recommendation

In [101]:
llama_avg_rank = np.mean(llama_ranks['sustainability_ranks'].apply(lambda x: x[0]['s-fairness-rank'] if len(x) else 0))
mistral_avg_rank = np.mean(mistral_ranks['sustainability_ranks'].apply(lambda x: x[0]['s-fairness-rank'] if len(x) else 0))

print(llama_avg_rank, mistral_avg_rank)

3.47 4.635


Mean Average S-Fairness Rank

In [102]:
def avg_rank(rec_cites):
    if len(rec_cites):
        return sum([x['s-fairness-rank'] for x in rec_cites])/len(rec_cites)
    return 0

llama_mar = np.mean(llama_ranks['sustainability_ranks'].apply(avg_rank))
mistral_mar = np.mean(mistral_ranks['sustainability_ranks'].apply(avg_rank))

print(llama_mar, mistral_mar)

4.029123015873015 5.427505952380952


The Mean Average Sustainability Rank for Llama is slightly lower for Llama as compared to Mistral 

### For how many prompts do the responses differ when sustainability is added? 

In [80]:
def count_sustainability_responses(df, len_th, comp = 'eq'):
    count_no_common_recs = 0

    for i, row in df.iterrows():
        l1 = set(row['rec_cities'])
        l2 = set(row['rec_cities_sustainable'])

        if comp == "eq":
            if len(l1 & l2) == len_th:
                count_no_common_recs += 1
        elif comp == "lte":
            if len(l1 & l2) <= len_th:
                count_no_common_recs += 1
        elif comp == "gte":
            if len(l1 & l2) >= len_th:
                count_no_common_recs += 1


    # print("Count of records where values of C are the same across different values of A:\n", count_same_values)
    return count_no_common_recs

In [81]:
llama_none_common = count_sustainability_responses(llama, 0)
mistral_none_common = count_sustainability_responses(mistral, 0)

In [82]:
print(llama_none_common, mistral_none_common)

0 0


For no prompt are the responses completely different when sustainability is added (i.e. a totally new list of recommended cities). 